In [2]:
import numpy as np
import tellurium as te
import scipy
import pandas as pd

In [3]:
model_desc = """
species $c1; species $l1; species $c4; species $l4; species $c2;

c1'=(1-(1-(K-c1-l1)/K-qH*l1/K)^n_hat)*p1_c*c1-(1-(1-(K-c1-l1)/K-qL*c1/K)^n_hat)*c1*qL*p1_l*l1/(K-l1-c1+qL*c1);
l1'=(1-(1-(K-c1-l1)/K-qL*c1/K)^n_hat)*p1_l*l1-(1-(1-(K-c1-l1)/K-qH*l1/K)^n_hat)*l1*qH*p1_c*c1/(K-l1-c1+qH*l1);

v1_l:=((1-(K-c1-l1)/K-qL*c1/K)^n_hat)*p1_l*l1 +(1-(1-(K-c1-l1)/K-qH*l1/K)^n_hat)*l1*qH*p1_c*c1/(K-l1-c1+qH*l1);
v1_c:=((1-(K-c1-l1)/K-qH*l1/K)^n_hat)*p1_c*c1 +(1-(1-(K-c1-l1)/K-qL*c1/K)^n_hat)*c1*qL*p1_l*l1/(K-l1-c1+qL*c1);

s:=1/(1+k_hat*c4)

a1_c:=a1_c_base*s
a2_c:=a2_c_base*s
a3_c:=a3_c_base*s
p1_c:=p1_c_base*s
p2_c:=p2_c_base*s

//c1+l1<K  //this is constraint

// initial l is 1 per kg of body weight
l1=4*(10^6)*bw 
l4=0 

// see figure 2D - minimum qL is 0.995
qH=0.005
 //this is to be estimated, giving a value arbitrarily
qL=0.995 // 1-ql=qh

bw=80
p1_l=15.2*10^(-3)
p3_l=0.69
p2_l=0.4

a1_l=1 // no info for cancer, I used the same with healthy
a2_l=0.57303 //no info for cancer, I used the same with healthy
a3_l=0.73994 //no info for cancer, I used the same with healthy
d4_l=0.01 //no info for cancer, I used the same with healthy
// the parameters from the supplementary document

#c4=3.1*(10^11)*bw  

c1=5.8*(10^6)*bw 
c2_initial=2.5*(10^7)*bw
c3_initial = 3.1*(10^11)*bw;

p1_c_base=15.2*10^(-3)
p3_c=0.69
p2_c_base=0.4
k_hat=9.677*10^(-12) //this is khat need to understand the difference
#K=9.8*(10^6)*bw  //assuming all cells are healthy in the beginning
K=c1+l1
n1=30
n2=20
n_hat=10
a1_c_base=1
a2_c_base=0.57303
a3_c_base=0.73994
//k=2.9222*10^(-13)
d4_c=0.01

   
"""
n1=30
n2=20
progenitor_sum_string1 = f"v2_l := 2*p2_l*l2_{n1} + 2*(1-a2_l)*p2_l*("
progenitor_sum_string2 = f"v2_c := 2*p2_c*c2_{n1} + 2*(1-a2_c)*p2_c*("
for i in range(n1+1):
    if i == 0:
        model_desc += "l2_0' = v1_l - p2_l*l2_0;\n"
        model_desc += "c2_0' = v1_c - p2_c*c2_0;\n"
        progenitor_sum_string1 += "l2_0"
        progenitor_sum_string2 += "c2_0"
    else:
        model_desc += f"l2_{i}' = 2*a2_l*p2_l*l2_{i-1} - p2_l*l2_{i};\n"
        model_desc += f"c2_{i}' = 2*a2_c*p2_c*c2_{i-1} - p2_c*c2_{i};\n"
        if i < n1:
            progenitor_sum_string1 += f"+ l2_{i}"
            progenitor_sum_string2 += f"+ c2_{i}"

progenitor_sum_string1 += ");\n"
progenitor_sum_string2 += ");\n"

precursor_sum_string1 = f"v3_l := 2*p3_l*l3_{n2} + 2*(1-a3_l)*p3_l*("
precursor_sum_string2 = f"v3_c := 2*p3_c*c3_{n2} + 2*(1-a3_c)*p3_c*("


for i in range(n2+1):
    if i == 0:
        model_desc += "l3_0' = v2_l - p3_l*l3_0;\n"
        model_desc += "c3_0' = v2_c - p3_c*c3_0;\n"
        precursor_sum_string1 += "l3_0"
        precursor_sum_string2 += "c3_0"
    else:
        model_desc += f"l3_{i}' = 2*a3_l*p3_l*l3_{i-1} - p3_l*l3_{i};\n"
        model_desc += f"c3_{i}' = 2*a3_c*p3_c*c3_{i-1} - p3_c*c3_{i};\n"
        if i < n2:
            precursor_sum_string1 += f"+ l3_{i}"
            precursor_sum_string2 += f"+ c3_{i}"

precursor_sum_string1 += ");\n"
precursor_sum_string2+= ");\n"

post_mitotic1="l4' = v3_l - d4_l*l4;\n"
post_mitotic2="c4' = v3_c - d4_c*c4;\n"

model_desc += progenitor_sum_string1
model_desc += progenitor_sum_string2

model_desc += precursor_sum_string1
model_desc += precursor_sum_string2

model_desc += post_mitotic1
model_desc += post_mitotic2


c2_str="c2:="
c3_str="c3:="

l2_str="l2:="
l3_str="l3:="


for i in range(n1+1):
    model_desc+=f"species $l2_{i};\n"
    model_desc+=f"l2_{i}=0;\n" 
    model_desc+=f"species $c2_{i};\n"
    model_desc+=f"c2_{i}=c2_initial/{n1+1};\n" 


    if i==0:
        c2_str+=f"c2_{i}"
        l2_str+=f"l2_{i}"
    else:
        c2_str+=f"+c2_{i}"
        l2_str+=f"+l2_{i}"

c2_str=c2_str +";\n"
l2_str=l2_str +";\n"

for i in range(n2+1):
    model_desc+=f"species $l3_{i};\n"
    model_desc+=f"l3_{i}=0;\n" 
    model_desc+=f"species $c3_{i};\n"
    model_desc+=f"c3_{i}=c3_initial/{n2+1};\n"

    if i==0:
        c3_str+=f"c3_{i}"
        l3_str+=f"l3_{i}"

    else:
        c3_str+=f"+c3_{i}"
        l3_str+=f"+l3_{i}"


c3_str=c3_str +";\n"
l3_str=l3_str +";\n"
     
model_desc+=c2_str 
model_desc+=c3_str 
     
model_desc+=l2_str 
model_desc+=l3_str 
model_desc+="blast:=l1+l2+l3+l4;\n"
model_desc += 'blast_percentage := (l1+l2+l3+l4)/(l1+l2+l3+l4+c1+c2+c3+c4);\n'


#=for i in range(n1+1):
    

r = te.loada(model_desc)

In [4]:
blood_counts = pd.read_excel('~/Desktop/AML_Helsinki_Data/Ven_blood_counts_16042023.xlsx', sheet_name='Blood_counts')
bm_blasts = pd.read_excel('~/Desktop/AML_Helsinki_Data/Ven_blood_counts_16042023.xlsx', sheet_name='Bone_marrow_blasts')
#cycle_days = pd.read_excel('~/Desktop/AML_Helsinki_Data/Ven_blood_counts_16042023.xlsx', sheet_name='Cycle_days')
#patient_data = pd.read_excel('~/Desktop/AML_Helsinki_Data/ven_responses_052023_open.xlsx')

In [5]:
blast_perc=bm_blasts.loc[bm_blasts["Event"]=="screening",["Pseudonym","bm_blasts"]]

In [7]:
healthy_counts=blood_counts.loc[(blood_counts["Event"] =="screening")& (blood_counts["Form"]=="lab_results"),  ["Pseudonym","b_neut", "b_eryt",	"b_trom" ]]

In [8]:
healthy_counts["b_eryt"]=healthy_counts["b_eryt"] *1000

In [10]:
healthy_counts['healthy_sum']=healthy_counts["b_neut"] + healthy_counts["b_eryt"] + healthy_counts["b_trom"]

In [11]:
patient_data=pd.merge(healthy_counts, blast_perc, on="Pseudonym", how="inner")

In [18]:
patient_data

,Pseudonym,b_neut,b_eryt,b_trom,healthy_sum,bm_blasts
0,8013,0.04,2750.0,35.0,2785.04,77.0
1,3028,2.30,2340.0,201.0,2543.30,35.0
2,1034,1.27,2820.0,127.0,2948.27,48.0
3,4046,0.20,3000.0,43.0,3043.20,46.0
4,5055,1.79,3300.0,45.0,3346.79,19.0
...,...,...,...,...,...,...
96,2104,0.31,3160.0,120.0,3280.31,21.0
97,8106,0.14,3630.0,34.0,3664.14,30.0
98,9109,0.19,3000.0,117.0,3117.19,80.0
99,3110,0.08,2920.0,77.0,2997.08,72.0


In [20]:
def run_parameter_scan(patient_id, initial_blast_percentage, initial_healthy_cells):
    
    """
    Args:
        initial_blast_percentage: between 0 and 1
        initial_healthy_cells: same units as in venex data
    """
    result = None
    results=pd.DataFrame(columns=["Parameter", "Value", "MinError"])
    healthy_cells_count = initial_healthy_cells*10**9*5
    if initial_blast_percentage > 1:
        initial_blast_percentage = initial_blast_percentage/100
    model = te.loada(model_desc)
    model.resetToOrigin()
    qL_values = np.linspace(0, 0.9999, 101)
    for qL in qL_values:
        sum_error_ql=[]
        if qL == 0:
            continue
        model.resetToOrigin()
        model.setValue('qL', qL)
        model.setValue('qH', 1-qL)
        result = model.simulate(0, 1000, 1000, selections=['time', 'blast_percentage', 'c4'])
        # testing if the parameter works
        for i in range(len(result)):
           sum_error_ql.append(np.abs(result['blast_percentage'][i] - initial_blast_percentage)/ initial_blast_percentage+\
            np.abs(result['c4'][i] - healthy_cells_count)/healthy_cells_count)
        tm=pd.DataFrame({'PatientID':patient_id,'Parameter':'qL','Value':qL, 'MinError':min(sum_error_ql)},index=[0])
        results=pd.concat([results,tm], ignore_index=True)
    # step 3: parameter sweep for p1_l
    print('Patient is not compatible with p1_l = p1_c, setting qL to 0.9999 and finding p1_l')
    p1_l_values = np.linspace(0.0, 0.9999, 101)
    for p1_l in p1_l_values:
        sum_error_p1_l=[]
        if p1_l == 0:
            continue
        model.resetToOrigin()
        model.setValue('qL', 0.9999)
        model.setValue('qH', 1-qL)
        model.setValue('p1_l', p1_l)
        result = model.simulate(0, 1000, 1000, selections=['time', 'blast_percentage', 'c4'])

        for i in range(len(result)):
            sum_error_p1_l.append(np.abs(result['blast_percentage'][i] - initial_blast_percentage)/ initial_blast_percentage+\
            np.abs(result['c4'][i] - healthy_cells_count)/healthy_cells_count)
        tm=pd.DataFrame({'PatientID':patient_id,'Parameter':'p1_l','Value':p1_l, 'MinError':  min(sum_error_p1_l)}, index=[0])
        results=pd.concat([results,tm], ignore_index=True)

    return results

In [22]:
patient_ids=patient_data["Pseudonym"].unique()

In [206]:
patient_id= round(pp['Pseudonym']);
patient_id

2111

In [154]:
#patient_data["Pseudonym"].duplicated().any()

False

In [26]:
all_results=pd.DataFrame(columns=['PatientID','Parameter','Value', 'MinError'])
for i in range(len(patient_ids)):
    print(patient_ids[i]);
    pp=patient_data.iloc[i,:];
    patient_id= round(pp['Pseudonym']);
    initial_blast_percentage=pp['bm_blasts'];
    initial_healthy_cells=pp['healthy_sum'];
    try:
        df2=run_parameter_scan(patient_id, initial_blast_percentage, initial_healthy_cells)
        all_results=pd.concat([all_results, df2], ignore_index=True)
    except:
        print("failed")


    

8013


/var/folders/g0/jjrpnby54x7gpjctb3x9clhw0000gp/T/ipykernel_80499/592941103.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Patient is not compatible with p1_l = p1_c, setting qL to 0.9999 and finding p1_l


/var/folders/g0/jjrpnby54x7gpjctb3x9clhw0000gp/T/ipykernel_80499/2188946461.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/g0/jjrpnby54x7gpjctb3x9clhw0000gp/T/ipykernel_80499/592941103.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



3028
Patient is not compatible with p1_l = p1_c, setting qL to 0.9999 and finding p1_l
1034


...

Patient is not compatible with p1_l = p1_c, setting qL to 0.9999 and finding p1_l
3110


/var/folders/g0/jjrpnby54x7gpjctb3x9clhw0000gp/T/ipykernel_80499/592941103.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Patient is not compatible with p1_l = p1_c, setting qL to 0.9999 and finding p1_l
2111


/var/folders/g0/jjrpnby54x7gpjctb3x9clhw0000gp/T/ipykernel_80499/592941103.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Patient is not compatible with p1_l = p1_c, setting qL to 0.9999 and finding p1_l


In [28]:
all_results.to_csv("parameter2_scan.csv")

In [242]:
pwd

'/Users/bahartercan/Desktop/HSC_LSC_final'